In [33]:
pip install -q "tensorflow-text==2.8.*"

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.54.1 requires numpy<1.21,>=1.17, but you have numpy 1.22.3 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [157]:
#Libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np
from tensorflow.keras.layers import TextVectorization

ImportError: cannot import name 'DepthwiseConv1D' from 'keras.layers.convolutional' (/opt/conda/lib/python3.9/site-packages/keras/layers/convolutional.py)

In [88]:
#Open data
data_file = open('Data.txt').read()
data_file

'The important thing is not to stop questioning. Curiosity has its own reason for existence. '

In [89]:
#Create tokenizer object
tokenizer = Tokenizer()

#Convert data to lowercase
data = data_file.lower().split('.')
data

['the important thing is not to stop questioning',
 ' curiosity has its own reason for existence',
 ' ']

In [151]:
#Create dictionary of words with the frequency they occur
#Every word gets unique value > 0
#0 is reserved for padding
tokenizer.fit_on_texts(data)

#Transforms sentences into set of integers from the dictionary
sequences = tokenizer.texts_to_sequences(data)

#Counts total words
total_words = len(tokenizer.word_index) + 1

#Prints it all out
print(tokenizer.word_index, '\n')
print(sequences, '\n')
print(total_words, '\n')

{'the': 1, 'important': 2, 'thing': 3, 'is': 4, 'not': 5, 'to': 6, 'stop': 7, 'questioning': 8, 'curiosity': 9, 'has': 10, 'its': 11, 'own': 12, 'reason': 13, 'for': 14, 'existence': 15} 

[[1, 2, 3, 4, 5, 6, 7, 8], [9, 10, 11, 12, 13, 14, 15], []] 

16 



In [152]:
#Pad sequences
tf.keras.preprocessing.sequence.pad_sequences(sequences, maxlen = 20, padding ='post', value = 0)

array([[ 1,  2,  3,  4,  5,  6,  7,  8,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0],
       [ 9, 10, 11, 12, 13, 14, 15,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0]], dtype=int32)

In [155]:
sequences = np.asarray(sequences)
sequences = tf.convert_to_tensor(sequences)
sequences.shape

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type list).

In [148]:
#Convert to embedding vectors
embedding_model = tf.keras.Sequential()
embedding_model.add(tf.keras.layers.Embedding(total_words, 512, input_length = 20))
embedding_model.compile('rmsprop', 'mse')
embedding_model.fit(sequences, epochs = 10)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type list).

In [ ]:
#Calculate positional encoding